INSTALLING NECESSARY LIBRARIES

In [1]:
#!pip install transformers

In [22]:
# !pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.2 MB/s eta 0:00:00


In [20]:
# !pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.0 MB/s eta 0:00:00


In [2]:
#!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


EXPORTING A SIMPLE ENGLISH-FRENCH DATASET FROM THE DATASETS

In [3]:
from datasets import load_dataset

dataset = load_dataset("opus_books", "en-fr")
train_data = dataset["train"]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/127085 [00:00<?, ? examples/s]

In [5]:
train_data

Dataset({
    features: ['id', 'translation'],
    num_rows: 127085
})

In [13]:
train_data["translation"][1]

{'en': 'Alain-Fournier', 'fr': 'Alain-Fournier'}

In [14]:
[translation["en"] for translation in train_data["translation"]]

['The Wanderer',
 'Alain-Fournier',
 'First Part',
 'I',
 'THE BOARDER',
 'He arrived at our home on a Sunday of November, 189-.',
 "I still say 'our home,' although the house no longer belongs to us.",
 'We left that part of the country nearly fifteen years ago and shall certainly never go back to it.',
 "We were living in the building of the Higher Elementary Classes at Sainte-Agathe's School.",
 "My father, whom I used to call M. Seurel as did other pupils, was head of the Middle School and also of the Higher Elementary classes where pupils worked for the preliminary teacher's examination.",
 'My mother taught the infants.',
 'At the extreme end of the small town, a long red house with five glass doors and a Virginia creeper upon its walls ; an immense courtyard with shelters, a washhouse and a huge gateway, on the side looking towards the village; on the north side, a small gate opening on the road leading to the station three kilometres off; on the south, at the back of the house,

SELECTING A PRETRAINED TOKENIZER FROM THE TRANSFORMERS

In [15]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")

def preprocess_function(examples):
    inputs = [translation["en"] for translation in examples["translation"]]
    targets = [translation["fr"] for translation in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length").input_ids
    model_inputs["labels"] = labels
    return model_inputs


tokenized_data = train_data.map(preprocess_function, batched=True)


/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Map:   0%|          | 0/127085 [00:00<?, ? examples/s]

SELECTING AN PRETRAINED LLM FOR THE TRAINING

In [18]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments

model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-fr")

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",  # Disable evaluation
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    save_total_limit=2,
    predict_with_generate=True,
    fp16=True,
)


from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-18-a21a51602926>:33: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
500,1.005900
1000,0.709100
1500,0.643100
2000,0.599500
2500,0.570600
3000,0.552900
3500,0.548700
4000,0.531800
4500,0.520600
5000,0.517000


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59513]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=23829, training_loss=0.46033716215789783, metrics={'train_runtime': 4437.3246, 'train_samples_per_second': 85.92, 'train_steps_per_second': 5.37, 'total_flos': 1.292392112062464e+16, 'train_loss': 0.46033716215789783, 'epoch': 3.0})

EVALUATING THE MODELS PERFORMANCE WITH A NEW DATA

In [24]:
import evaluate

bleu = evaluate.load("sacrebleu")

def compute_bleu(predictions, references):
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_refs = [[ref] for ref in references]
    return bleu.compute(predictions=decoded_preds, references=decoded_refs)

test_sentences = ["This is a test sentence.", "The AI is fine-tuned for translation."]

inputs = tokenizer(test_sentences, return_tensors="pt", padding=True, truncation=True).input_ids.to("cuda")

outputs = model.generate(inputs, max_length=50)

predictions = tokenizer.batch_decode(outputs, skip_special_tokens=True)

# Example reference translations (e.g., French translations for the test sentences)
references = ["C'est une phrase de test.", "L'IA est bien fine-tunée pour la traduction."]

bleu_score = compute_bleu(outputs, references)

print(f"Predictions: {predictions}")
print(f"BLEU score: {bleu_score['score']}")


Predictions: ['Voilà une phrase épreuvée.', "L'aile est fine à traduire."]
BLEU score: 8.822674259118783


A BLEU score of 8.82 indicates that the model's translations are not very accurate compared to the reference translations. Ideally, a score of 30-40 suggests better performance on high-quality data. This low score may stem from insufficient training data or suboptimal hyperparameters. Improving data quality and tuning the model can help enhance translation accuracy.

CONCLUSION :
1. The low BLEU score (8.82) indicates that model is struggling to generate accurate translations.
2. The model might still need more training data or more training time to improve its performance.